In [1]:
import requests
from bs4 import BeautifulSoup
import time
import numpy as np
import urllib
import os
from selenium import webdriver
from time import sleep
import pandas as pd

### Job searching - My career futures

In [2]:
url = ["https://www.mycareersfuture.sg/search?search=data%20scientist&page={}", # data scientist 
       "https://www.mycareersfuture.sg/search?search=business%20intelligence&page={}", # business intelligence
       "https://www.mycareersfuture.sg/search?search=data%20analyst&page={}", # data analyst
       "https://www.mycareersfuture.sg/search?search=machine%20learning&page={}", # machine learning,
       "https://www.mycareersfuture.sg/search?search=business%20analyst&page={}", # business analyst page
       "https://www.mycareersfuture.sg/search?search=data%20engineer&page={}", #data engineer
       "https://www.mycareersfuture.sg/search?search=artificial%20intelligence&page={}",# artificial intelligence
       "https://www.mycareersfuture.sg/search?search=Data%20Architect&page={}", # Data Architect
       "https://www.mycareersfuture.sg/search?search=Research%20Scientist&page={}", # Resarch Scientist Architect
       "https://www.mycareersfuture.sg/search?search=Finance%20Analyst&page={}" # Finance Analyst
      ]

job_urls = []

# iterate the url and selenium the html
for links in url:
    # iterate 0 to 20 pages 
    for pages in range(0,20+1): 
        #Controls the ChromeDriver and allows you to drive the browser.
        driver = webdriver.Chrome(executable_path="/Users/ivans/Desktop/GA/chromedriver.exe")
    
        #Loads a web page in the current browser session
        driver.get(links.format(pages))
    
        # Delay execution for a given number of seconds.
        sleep(4)
    
        # Defines the basic interface called by the tree builders.
        html = BeautifulSoup(driver.page_source, 'lxml')
        # iterate through job cards and get relevant details (https://stackoverflow.com/questions/2830530/matching-ids-in-beautifulsoup)
        for job_card in html.find_all('div', id=lambda x: x and x.startswith('job-card-')):
            job_urls.append('https://www.mycareersfuture.sg'+job_card.find('a')['href'])
      
        # close the webspage after ger ther relavant details  
        driver.close()


In [3]:
len(job_urls)

1242

In [4]:
# turn the list to set due to the set removes all duplicate elements
job_urls2 = set(job_urls)

In [5]:
# turn the sets to the list again then convert it to dataframe then csv
job_df=pd.DataFrame(list(job_urls2))

In [6]:
# open the csv
# job_df_DS = pd.read_csv('job_url2_DS.csv')
# get the value of the job site columns
job_ls = job_df.iloc[:,0].values

In [7]:
#Create lists for information needed
companies = []
titles = []
addresses = []
emp_type = []
seniority = []
category = []
salaries = []
responsibilities = []
requirements = []
failed_url = []

# Controls the ChromeDriver and allows you to drive the browser.
driver = webdriver.Chrome(executable_path="/Users/ivans/Desktop/GA/chromedriver.exe")

# In order to iterate every columns by its index, we return an enumerate object.
for index, url_job in enumerate(job_ls[0:1070]):
     
    try:
        # iterate the web page from the list (job_ls)
        #Loads a web page in the current browser session
        driver.get(url_job)
        # Delay execution for a given number of seconds.
        sleep(4)
        html_job = driver.page_source
        # Defines the basic interface called by the tree builders
        soup_job = BeautifulSoup(html_job, "lxml")
        # Append the related information to 
        companies.append(soup_job.find('p', {'id': 'company'}))
        titles.append(soup_job.find('h1', {'id': 'job_title'}))
        addresses.append(soup_job.find('p', {'id': 'address'}))    
        emp_type.append(soup_job.find('p', {'id': 'employment_type'}))
        seniority.append(soup_job.find('p', {'id': 'seniority'}))
        category.append(soup_job.find('p', {'id': 'job-categories'}))
        salaries.append(soup_job.find('div', {'class': 'lh-solid'}))
        responsibilities.append(soup_job.find('div',{'id':'job_description'}))
        requirements.append(soup_job.find('div',{'id':'requirements'}))
    except:
        failed_url.append(url_job)

In [8]:
# Define the sequence and structure of the dataframe
lists = [companies, titles, addresses, emp_type, seniority, category, salaries, responsibilities, requirements]
# Create a dictionary
clean = {}

# In order to iterate every columns by its index, we return an enumerate object.
for index, items in enumerate(lists):
    # Create a list for the dictionary, keys = index, values = items we gonna to append
    clean[index] = []
    # We use the try and except to append the text of the elements (excluding the html code). If not, the list index will be out of range 
    for elements in items:
        try:
            clean[index].append(elements.text)
        except:
            clean[index].append(None)
            
# Convert dictionary into dataframe
df_job_mycareersfuture = pd.DataFrame(clean) 
# Rename columns
df_job_mycareersfuture.columns = ['titles', 'companies', 'locations', 'category', 'emp_type', 'seniority', 'salaries', 'responsibilities', 'requirements']

In [9]:
df_job_mycareersfuture.shape

(1012, 9)

In [10]:
df_job_mycareersfuture.to_csv('df_job_mycareersfuture.csv', encoding='utf-8')